In [1]:
import codecs
import pandas as pd

In [2]:
nomLexique3="/Users/gilles/Dropbox/Travail/Recherche/BoKu/LingInfo/Lexique3/Lexique381/Lexique381.csv"

In [3]:
dfLex3Brut=pd.read_csv(nomLexique3,sep="\t")

In [4]:
lex3Categories=[c for c in dfLex3Brut["4_cgram"].unique().tolist() if isinstance(c,str)]
print ", ".join(sorted(lex3Categories))

ADJ, ADJ:dem, ADJ:ind, ADJ:int, ADJ:num, ADJ:pos, ADV, ART:def, ART:ind, AUX, CON, LIA, NOM, ONO, PRE, PRO:dem, PRO:ind, PRO:int, PRO:per, PRO:pos, PRO:rel, VER


In [5]:
dfNoms=dfLex3Brut[dfLex3Brut["4_cgram"]=="NOM"].loc[:,"1_ortho":"6_nombre"]
dfNoms.count()["1_ortho"]

48286

In [6]:
dfAdjectifs=dfLex3Brut[dfLex3Brut["4_cgram"]=="ADJ"].loc[:,"1_ortho":"6_nombre"]
dfAdjectifs.count()["1_ortho"]

26806

In [7]:
dfNoms.groupby("5_genre").count()
dfNoms.groupby("6_nombre").count()
dfNoms.groupby(["5_genre","6_nombre"]).count()[["3_lemme"]]

3_lemme
5_genre 6_nombre         
f       p            7069
        s           11787
m       p            9673
        s           15351

In [9]:
dfNoms[dfNoms["5_genre"].isnull()].head()

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre
271,abolitionniste,abolisjonist,abolitionniste,NOM,NaN,s
273,abolitionnistes,abolisjonist,abolitionniste,NOM,NaN,p
357,aborigène,aboRiZEn,aborigène,NOM,NaN,s
359,aborigènes,aboRiZEn,aborigène,NOM,NaN,p
378,aboulique,abulik,aboulique,NOM,NaN,s


In [13]:
dfNoms[dfNoms["5_genre"]=="m"].head()

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre
0,a,a,a,NOM,m,NaN
12,a priori,apRijoRi,a priori,NOM,m,NaN
13,aa,aa,aa,NOM,m,s
17,abaca,abaka,abaca,NOM,m,s
27,abaisse-langue,abEsl@g,abaisse-langue,NOM,m,NaN


# Combinaisons de traits

In [14]:
def combinaisonsTraits(dfLocal):
    genres={}
    result={}
    for g in "mf":
        genres[g]=dfLocal[dfLocal["5_genre"]==g]
    genres["nan"]=dfLocal[dfLocal["5_genre"].isnull()]
    for g in genres:
        for n in "sp":
            result[(g,n)]=genres[g][genres[g]["6_nombre"]==n]
        result[(g,"nan")]=genres[g][genres[g]["6_nombre"].isnull()]
    return result

In [55]:
def calculerCouvertureNoms(dfLocal):
    nbFormes=0
    for k,v in combinaisonsTraits(dfLocal).iteritems():
#        print k,v.count()["3_lemme"]
        if not "nan" in k:
            nbFormes+=v.count()["3_lemme"]
        else:
            if k==("nan","nan"):
                nbFormes+=4*v.count()["3_lemme"]
            else:
                nbFormes+=2*v.count()["3_lemme"]
#    len(pd.unique(dfNoms[["3_lemme","5_genre"]].values.ravel("K")))

    dfEpicenes=dfLocal[dfLocal["5_genre"].isnull()]
    dfGenres=dfLocal[~dfLocal["5_genre"].isnull()]

    nbEpicenes=len(pd.unique(dfEpicenes[["3_lemme","5_genre"]].values.ravel("K")))
    nbGenres=len(pd.unique(dfGenres[["3_lemme","5_genre"]].values.ravel("K")))
    nbTotal=nbEpicenes*2+nbGenres

    print u"épicènes\t%d (x2)"%nbEpicenes
    print u"genrés\t\t%d"%nbGenres
    print u"nombre total\t%d"%nbTotal
    print 
    print "nombre de formes de noms",nbFormes
    print "nb de cases de paradigme",nbTotal*2
    print u"couverture %0.3g%%"%(100*float(nbFormes)/(nbTotal*2))

In [56]:
calculerCouvertureNoms(dfNoms)

épicènes	1667 (x2)
genrés		28957
nombre total	32291

nombre de formes de noms 52708
nb de cases de paradigme 64582
couverture 81.6%


In [53]:
def calculerCouvertureAdjectifs(dfLocal):
    nbFormes=0
    for k,v in combinaisonsTraits(dfLocal).iteritems():
#        print k,v.count()["3_lemme"]
        if not "nan" in k:
            nbFormes+=v.count()["3_lemme"]
        else:
            if k==("nan","nan"):
                nbFormes+=4*v.count()["3_lemme"]
            else:
                nbFormes+=2*v.count()["3_lemme"]
    nbTotal=len(pd.unique(dfLocal[["3_lemme"]].values.ravel("K")))
    print u"nombre total d'adjectifs",nbTotal
    print
    print "nombre de formes d'ajectifs",nbFormes
    print "nombre de cases de paradigme",nbTotal*4
    print u"couverture %0.3g%%"%(100*float(nbFormes)/(nbTotal*4))

In [54]:
calculerCouvertureAdjectifs(dfAdjectifs)

nombre total d'adjectifs 11580

nombre de formes d'ajectifs 32568
nombre de cases de paradigme 46320
couverture 70.3%


# Calculer le nombre de microclasses